In [ ]:
# | default_exp cli

In [ ]:
# | export

from typing import *

import typer
from pathlib import Path

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.app_description_validator import (
    validate_app_description,
)
from faststream_gen._code_generator.helper import (
    set_logger_level,
    add_tokens_usage,
    get_relevant_prompt_examples,
    strip_white_spaces,
    write_file_contents,
    ensure_openai_api_key_set,
)
from faststream_gen._code_generator.constants import MODEL_PRICING, EMPTY_DESCRIPTION_ERROR, OpenAIModel
from faststream_gen._components.new_project_generator import create_project

In [ ]:
from typer.testing import CliRunner
from tempfile import TemporaryDirectory

import pytest

from faststream_gen._components.logger import suppress_timestamps
from faststream_gen._code_generator.helper import mock_openai_create

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

app = typer.Typer(
    short_help="Commands for accelerating FastStream project creation using advanced AI technology",
     help="""Commands for accelerating FastStream project creation using advanced AI technology.

These commands use OpenAI's model to generate FastStream project. To access this feature, kindly sign up if you haven't already and create an API key with OpenAI. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

Once you have the key, please set it in the OPENAI_API_KEY environment variable before executing the code generation commands.

Note: Accessing OpenAI API incurs charges. For further information on pricing and free credicts, check this link: https://openai.com/pricing
    """,
)

In [ ]:
# | export


def _calculate_price(total_tokens_usage: Dict[str, int], model: str) -> float:
    """Calculates the total price based on the number of promt & completion tokens and the models price for input and output tokens (per 1k tokens).

    Args:
        total_tokens_usage: OpenAI "usage" dictionaries which defines prompt_tokens, completion_tokens and total_tokens


    Returns:
        float: The price for used tokens
    """
    model_price = MODEL_PRICING[model]
    price = (total_tokens_usage["prompt_tokens"] * model_price["input"] + total_tokens_usage["completion_tokens"] * model_price["output"]) / 1000
    return price

In [ ]:
usage = {"prompt_tokens": 129, "completion_tokens": 1, "total_tokens": 130}

usage_list = [usage, usage]
total_tokens_usage = add_tokens_usage(usage_list)

actual = _calculate_price(total_tokens_usage, "gpt-4")
expected = 0.007859999999999999

print(actual)
assert actual == expected

0.007859999999999999


In [ ]:
# | export



def _get_description(input_path: str) -> str:
    """Reads description from te file and returns it as a string

    Args:
        input_path: Path to the file with the desription
        
    Raises:
        ValueError: If the file does not exist.

    Returns:
        The description string which was read from the file.
    """
    try:
        with open(input_path) as file:
            # Read all lines to list
            lines = file.readlines()
            # Join the lines 
            description = '\r'.join(lines)
            logger.info(f"Reading application description from '{str(Path(input_path).absolute())}'.")
    except Exception as e:
        raise ValueError(f"Error while reading from the file: '{str(Path(input_path).absolute())}'\n{str(e)}")
    return description

In [ ]:
with TemporaryDirectory() as d:
    input_path = f"{d}/app_description.txt"
    description = """Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.

It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.
For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.

Use SASL_SSL with SCRAM-SHA-256 for authentication with username and password."""
    
    
    write_file_contents(input_path, description)
    actual = _get_description(input_path)
    display(actual)
    expected = "Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.\n\r\n\rIt should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.\n\rFor each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.\n\r\n\rUse SASL_SSL with SCRAM-SHA-256 for authentication with username and password."         
    
    assert actual == expected, actual

[INFO] __main__: Reading application description from '/tmp/tmp_8r2wav6/app_description.txt'.


"Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number.\n\r\n\rIt should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.\n\rFor each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.\n\r\n\rUse SASL_SSL with SCRAM-SHA-256 for authentication with username and password."

In [ ]:
with pytest.raises(ValueError) as e:
        _get_description("incorrect_path")
        
print(e.value)
assert str(e.value) == f"Error while reading from the file: '{str(Path('incorrect_path').absolute())}'\n[Errno 2] No such file or directory: 'incorrect_path'"

Error while reading from the file: '/work/fastkafka-gen/nbs/incorrect_path'
[Errno 2] No such file or directory: 'incorrect_path'


In [ ]:
# | export


def _validate_app_description(
    description: Optional[str],
    input_path: str,
    model: OpenAIModel,
    tokens_list: List[Dict[str, int]],
) -> Tuple[str, List[Dict[str, int]]]:
    if not description:
        if not input_path:
            raise ValueError(EMPTY_DESCRIPTION_ERROR)
        description = _get_description(input_path)
        cleaned_description = strip_white_spaces(description)
        return validate_app_description(cleaned_description, model.value, tokens_list)

In [ ]:
with pytest.raises(ValueError) as e:
    _validate_app_description(None, None, OpenAIModel.gpt3, [])

In [ ]:
test_response = "3"

with mock_openai_create(test_response):
    _validate_app_description("some valid description", None, OpenAIModel.gpt3, [])

In [ ]:
test_response = "3"

with TemporaryDirectory() as d:
    input_path = Path(d)/"input.txt"
    with open(input_path, "w", encoding="utf-8") as f:
        f.write("app description")
    
    with mock_openai_create(test_response):
        _validate_app_description(None, str(input_path), OpenAIModel.gpt3, [])

[INFO] __main__: Reading application description from '/tmp/tmp43mtmta8/input.txt'.
✨  Generating a new FastStream application!
[INFO] faststream_gen._code_generator.app_description_validator: ==== App description validation ====
⠹ Validating the application description... [INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

In [ ]:
# | export


@app.command(
    "generate",
    help="Effortlessly create a new FastStream project based on the app description.",
)
@set_logger_level
def generate_fastkafka_app(
    description: Optional[str] = typer.Argument(
        None,
        help="""Summarize your FastStream application in a few sentences!


\nInclude details about messages, topics, servers, and a brief overview of the intended business logic.


\nThe simpler and more specific the app description is, the better the generated app will be. Please refer to the below example for inspiration:


\nCreate a FastStream application using localhost broker for testing and use the default port number. 
It should consume messages from the "input_data" topic, where each message is a JSON encoded object containing a single attribute: 'data'. 
For each consumed message, create a new message object and increment the value of the data attribute by 1. Finally, send the modified message to the 'output_data' topic.
\n""",
    ),
    input_path: str = typer.Option(
        None,
        "--input_file",
        "-i",
        help="""
        The path to the file with the app desription. This path should be relative to the current working directory.
        
        \n\nIf the app description is passed via both a --input_file and a command line argument, the description from the command line will be used to create the application.
        """,
    ),
    output_path: str = typer.Option(
        ".",
        "--output_path",
        "-o",
        help="The path to the output directory where the generated project files will be saved. This path should be relative to the current working directory.",
    ),
    model: OpenAIModel = typer.Option(
        OpenAIModel.gpt3.value,
        "--model",
        "-m",
        help=f"The OpenAI model that will be used to create the FastStream project. For better results, we recommend using '{OpenAIModel.gpt4.value}'.",
    ),
    verbose: bool = typer.Option(
        False,
        "--verbose",
        "-v",
        help="Enable verbose logging by setting the logger level to INFO.",
    ),
    save_log_files: bool = typer.Option(
        False,
        "--dev",
        "-d",
        help="Save the complete logs generated by faststream-gen inside the output_path directory.",
    ),
) -> None:
    """Effortlessly create a new FastStream project based on the app description."""
    logger.info("Project generation started.")
    try:
        tokens_list: List[Dict[str, int]] = []
        ensure_openai_api_key_set()
        
        # Step 1: Validate description
        validated_description, tokens_list = _validate_app_description(description, input_path, model, tokens_list)
        
        # Step 2: Project creation
        create_project(output_path)
        
        print("Project Created")
        
    except (ValueError, KeyError) as e:
        fg = typer.colors.RED
        typer.secho(e, err=True, fg=fg)
        raise typer.Exit(code=1)
    except Exception as e:
        fg = typer.colors.RED
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=fg)
        raise typer.Exit(code=1)
    finally:
        total_tokens_usage = add_tokens_usage(tokens_list)
        price = _calculate_price(total_tokens_usage, model.value)

        fg = typer.colors.CYAN
        typer.secho(f" Tokens used: {total_tokens_usage['total_tokens']}", fg=fg)
        logger.info(f"Prompt Tokens: {total_tokens_usage['prompt_tokens']}")
        logger.info(f"Completion Tokens: {total_tokens_usage['completion_tokens']}")
        typer.secho(f" Total Cost (USD): ${round(price, 5)}", fg=fg)

In [ ]:
result = runner.invoke(app, ["generate", "--help"])